# Higher-Order Functions en Databricks / Spark SQL

En **Spark SQL (Databricks)**, las **Higher-Order Functions (HOF)** son funciones que reciben **otras funciones (lambdas)** como argumento y operan sobre **colecciones complejas**, principalmente:

- `ARRAY`
- `MAP`

Son muy importantes para el examen **Databricks Data Engineer Associate**.

---

## Concepto Clave

Una higher-order function:

✔ Opera sobre arrays o maps  
✔ Usa una lambda (`x -> expresión`)  
✔ Evita `explode + group by` innecesarios  
✔ Permite transformaciones eficientes

Ejemplo de lambda:

```sql
x -> x * 2


### TRANSFORM

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW order_items AS
SELECT * FROM
VALUES
  (1, array('smartphone', 'laptop', 'monitor')),
  (2, array('tablet', 'headphones', 'smartwatch')),
  (3, array('keyboard', 'mouse'))
AS orders(order_id, items);



In [0]:
SELECT * FROM order_items;

In [0]:
SELECT order_id, TRANSFORM(items,x->upper(x)) as upper_items FROM order_items;

### Filter

In [0]:
SELECT order_id, 
    FILTER(items, x -> x LIKE '%smart%') as string_smart 
FROM order_items;

### FILTER: devuelve un valor verdadero o falso(TRUE, FALSE), si cumple una condicion

Queremos ver si el articulo  incluye  el nombre monitor

In [0]:
SELECT order_id, 
    EXISTS(items, x -> x LIKE '%monitor%') as string_smart 
FROM order_items;

In [0]:
CREATE OR REPLACE TEMP VIEW order_items AS
SELECT * FROM VALUES
(1, array(
    named_struct('name', 'smartphone', 'price', 699),
    named_struct('name', 'laptop', 'price', 1199),
    named_struct('name', 'monitor', 'price', 399)
)),
(2, array(
    named_struct('name', 'tablet', 'price', 599),
    named_struct('name', 'headphones', 'price', 199),
    named_struct('name', 'smartwatch', 'price', 299)
)),
(3, array(
    named_struct('name', 'keyboard', 'price', 89),
    named_struct('name', 'mouse', 'price', 59)
))
AS orders(order_id, items);



In [0]:
-- Para verificar el contenido:
SELECT * FROM order_items;

In [0]:
SELECT 
    order_id, 

   TRANSFORM(items, x -> UPPER(x["name"])) as upper_items

FROM order_items;

In [0]:
SELECT 
    order_id, 

   TRANSFORM(items, x -> named_struct
                          ('name',UPPER(x.name), 'price', round(x.price*1.10,2)) ) as upper_items

FROM order_items;

In [0]:
SELECT 
    order_id, 

   AGGREGATE(items,0,(acc,x)->acc+x.price) as total_order_price

FROM order_items;

# MAPS FUNTIONS

MAPS: un maps es un coleccion igual que un diccionario conformado por clave:valor
- TRANSFORM_VALUES
- TRANSFORM_KEYS
- MAP_FILTER

In [0]:
CREATE OR REPLACE TEMP VIEW order_item_prices AS
SELECT * FROM VALUES
(1, map('smartphone', 699, 'laptop', 1199, 'monitor', 399)),
(2, map('tablet', 599, 'headphones', 199, 'smartwatch', 299)),
(3, map('keyboard', 89, 'mouse', 59))
AS orders(order_id, item_prices);

-- Consulta para verificar
SELECT * FROM order_item_prices;

In [0]:
SELECT 
  transform_keys(item_prices, (k,v) -> upper(k))
FROM order_item_prices;

In [0]:
SELECT 
  transform_values(item_prices, (k,v) -> v*1.10)
FROM order_item_prices;

In [0]:
SELECT 
  map_filter(item_prices, (k,v) -> v>500)
FROM order_item_prices;